## Importar librerias y modulos

In [1]:
import requests
import json
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

/Users/julianmauriciocarocorrea/Desktop/Analisis_de_datos innovador/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


## Carga de datasets 
- Generacion
- Capacidad
- Demanda


In [5]:
df_generacion = pd.read_csv('../data/Generacion.csv')

#carga de dataset de demanda
dataset_id = '14fabb'              # Cambia este valor por el ID del dataset deseado
start_date = '2022-01-01'          # Fecha de inicio
end_date = '2025-11-05'            # Fecha de fin

df_demanda = descargar_datos_simem(dataset_id, start_date, end_date)
if df_demanda is not None:
    print(df_demanda.head())


ConnectionError: HTTPSConnectionPool(host='www.simem.co', port=443): Max retries exceeded with url: /backend-files/api/datos-publicos?datasetId=14fabb&startDate=2022-01-01&endDate=2025-11-05 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1532de780>: Failed to resolve 'www.simem.co' ([Errno 8] nodename nor servname provided, or not known)"))

## Visualzación de datos crudos

In [150]:
df_demanda

,CodigoVariable,FechaHora,CodigoSICAgente,TipoMercado,Version,Valor,UnidadMedida,CodigoDuracion
0,DdaReal,2024-11-21T23:00:00,VESC,No Regulado,TX7,1104.32,kWh,PT1H
1,DdaReal,2024-11-21T22:00:00,VESC,No Regulado,TX7,1097.19,kWh,PT1H
2,DdaReal,2024-11-21T21:00:00,VESC,No Regulado,TX7,1172.70,kWh,PT1H
3,DdaReal,2024-11-21T20:00:00,VESC,No Regulado,TX7,1294.84,kWh,PT1H
4,DdaReal,2024-11-21T19:00:00,VESC,No Regulado,TX7,1175.95,kWh,PT1H
...,...,...,...,...,...,...,...,...
3349603,DdaReal,2023-10-29T04:00:00,ITLC,Regulado,TX4,79.38,kWh,PT1H
3349604,DdaReal,2023-10-29T03:00:00,ITLC,Regulado,TX4,81.84,kWh,PT1H
3349605,DdaReal,2023-10-29T02:00:00,ITLC,Regulado,TX4,84.25,kWh,PT1H
3349606,DdaReal,2023-10-29T01:00:00,ITLC,Regulado,TX4,88.69,kWh,PT1H


In [151]:
df_generacion

,Fecha,CodigoPlanta,TipoGeneracion,TipoClasificacion,TipoDespachoRecurso,GeneracionRealEstimada,GeneracionProgramadaDespacho,GeneracionProgramadaRedespacho
0,2019-12-30,2T4P,Hidraulica,Hidraulica,No Despachado Centralmente,301801.06,354240.00,354240.00
1,2019-12-30,TFL1,Termica,Termica,Despachado Centralmente,0.00,0.00,0.00
2,2019-12-30,MNT1,Hidraulica,Hidraulica,No Despachado Centralmente,41952.37,43200.00,43200.00
3,2019-12-30,VNTA,Hidraulica,Hidraulica,No Despachado Centralmente,54232.03,40560.00,40560.00
4,2019-12-30,2VJS,Solar,Solar,No Despachado Centralmente,44690.88,44690.88,44690.88
...,...,...,...,...,...,...,...,...
1091675,2018-09-22,PPA2,Termica,Termica,Despachado Centralmente,0.00,0.00,0.00
1091676,2018-09-22,2TYI,Termica,AUTOGENERADOR,No Despachado Centralmente,8916.00,8000.00,8000.00
1091677,2018-09-22,INZ1,Hidraulica,Hidraulica,No Despachado Centralmente,11586.31,12000.00,12000.00
1091678,2018-09-22,RMR1,Hidraulica,Hidraulica,No Despachado Centralmente,33636.83,47000.00,47000.00


## Columnas de interes:
- df_generacion: Fecha y GeneracionRealEstimada (kW)
- df_demanda: AnioEstudio, MesEstudio y DemandaSIN (kW)

In [152]:
df_generacion_fil = df_generacion[['Fecha', 'GeneracionRealEstimada']]
df_demanda_fil = df_demanda[['FechaHora','Valor']]


In [153]:
df_demanda_fil.head()

,FechaHora,Valor
0,2024-11-21T23:00:00,1104.32
1,2024-11-21T22:00:00,1097.19
2,2024-11-21T21:00:00,1172.70
3,2024-11-21T20:00:00,1294.84
4,2024-11-21T19:00:00,1175.95


In [154]:
df_generacion_fil.describe()

,GeneracionRealEstimada
count,1.091680e+06
mean,8.379024e+05
std,2.778703e+06
min,0.000000e+00
25%,1.420000e+03
50%,3.925703e+04
75%,3.098320e+05
max,2.997673e+07


# Funciones


Función para convertir de AAAA-MM-DD a AAAA-MM: 
- Se toman los valores de GeneracionRealEstimada diarios y se suman a un total mensual

In [3]:
# Función para totalizar los valores por día
def preparar_datos_diarios(df, fecha_col='Fecha', valor_col='GeneracionRealEstimada'):
    """
    Agrupa los datos por día y suma todos los valores de cada fecha.
    Si hay múltiples registros por día (por ejemplo, por hora), los totaliza.
    
    - df: DataFrame con columna de fecha y valores.
    - fecha_col: nombre de columna de fecha.
    - valor_col: nombre de columna con valores.
    
    Retorna un DataFrame con columnas: 'Fecha' (AAAA-MM-DD) y 'Valor' (suma diaria)
    """
    # Convertir columna de fecha a datetime
    df = df.copy()
    df[fecha_col] = pd.to_datetime(df[fecha_col])
    
    # Extraer solo la fecha (sin hora) para agrupar por día
    df['Fecha_Dia'] = df[fecha_col].dt.date
    
    # Agrupar por fecha y sumar todos los valores del mismo día
    resultado = df.groupby('Fecha_Dia')[valor_col].sum().reset_index()
    resultado.columns = ['Fecha', 'Valor']
    
    # Convertir de nuevo a datetime para mantener consistencia
    resultado['Fecha'] = pd.to_datetime(resultado['Fecha'])
    
    # Ordenar por fecha
    resultado = resultado.sort_values('Fecha').reset_index(drop=True)
    
    return resultado


# Función para normalizar los datos en formato Kw
def normalizar_a_gwh(df, valor_col='Valor'):
    df = df.copy()
    # Cambiar de MWh a GWh (dividir por 1000, no por 1e9)
    df['Valor_KWh'] = df[valor_col] / 1000  # O verificar unidades originales
    df = df[['Fecha', 'Valor_KWh']]
    return df

# función para descargar los datos por la API de SIMEM
def descargar_datos_simem(dataset_id, start_date, end_date):
    """
    Descarga y retorna un DataFrame con los datos del SiMEM según dataset_id y fechas.
    """
    url = (f"https://www.simem.co/backend-files/api/datos-publicos?"
           f"datasetId={dataset_id}&startDate={start_date}&endDate={end_date}")
    parameters = []
    buffer = ""
    # Solicitud POST con streaming
    with requests.post(url, json=parameters, stream=True) as response:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                buffer += chunk.decode('utf-8')
    try:
        data = json.loads(buffer)
        df = pd.DataFrame(data)
        return df
    except json.JSONDecodeError as e:
        print("Error al decodificar JSON:", e)
        return None

## Sumar a total mensual y nomalizar los datos a Gw

In [156]:
df_mensual_generacion = normalizar_a_gwh(preparar_datos_diarios(df_generacion_fil))
df_diario_demanda_gwh = normalizar_a_gwh(preparar_datos_diarios(df_demanda_fil, fecha_col='FechaHora',valor_col='Valor'))

In [157]:
df_diario_generacion_gwh = df_mensual_generacion[df_mensual_generacion['Fecha'] >= '2022'].reset_index(drop=True)
df_diario_generacion_gwh

,Fecha,Valor_KWh
0,2022-01-01,160426.13462
1,2022-01-02,169141.84867
2,2022-01-03,195492.50417
3,2022-01-04,202153.87807
4,2022-01-05,203409.91956
...,...,...
1377,2025-10-09,241913.58000
1378,2025-10-10,239768.29000
1379,2025-10-11,221022.76000
1380,2025-10-12,200215.82000


In [158]:
df_diario_demanda_gwh

,Fecha,Valor_KWh
0,2022-01-01,161861.12679
1,2022-01-02,170246.43165
2,2022-01-03,193038.31367
3,2022-01-04,198164.69436
4,2022-01-05,199556.70091
...,...,...
1400,2025-11-01,225063.21209
1401,2025-11-02,208303.42241
1402,2025-11-03,212521.73262
1403,2025-11-04,234495.84525


## Modelos de maching learning

In [159]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')

# ========================= PREPARACIÓN DE DATOS =========================
def preparar_datos_ml_diario(df, col_fecha='Fecha', col_valor='Valor_GWh'):
    """
    Prepara el DataFrame con datos diarios para modelos de ML.
    Convierte fechas a números enteros consecutivos (días desde inicio).
    """
    df = df.copy()
    df[col_fecha] = pd.to_datetime(df[col_fecha])
    df = df.sort_values(col_fecha).reset_index(drop=True)
    
    # Días desde la primera fecha
    fecha_inicio = df[col_fecha].min()
    df['dias_desde_inicio'] = (df[col_fecha] - fecha_inicio).dt.days
    
    X = df[['dias_desde_inicio']].values
    y = df[col_valor].values
    return X, y, df, fecha_inicio

def preparar_datos_prophet_diario(df, col_fecha='Fecha', col_valor='Valor_GWh'):
    """
    Prepara el DataFrame para Prophet (requiere columnas 'ds' y 'y').
    """
    df = df.copy()
    df['ds'] = pd.to_datetime(df[col_fecha])
    df['y'] = df[col_valor]
    return df[['ds', 'y']]

# ========================= PREDICCIÓN HASTA 2030 =========================
def predecir_hasta_2030_diario(model, df_prep, fecha_inicio, tipo='ml'):
    """
    Extiende predicciones diarias hasta 2030-12-31.
    """
    ultima_fecha = pd.to_datetime(df_prep['Fecha'].max())
    fecha_final = pd.Timestamp('2030-12-31')
    
    # Generar fechas futuras diarias
    fechas_futuras = pd.date_range(start=ultima_fecha + pd.Timedelta(days=1), 
                                    end=fecha_final, freq='D')
    
    if tipo == 'ml':
        # Para RF y LR
        ultimo_dia = df_prep['dias_desde_inicio'].max()
        dias_futuros = np.arange(ultimo_dia + 1, 
                                 ultimo_dia + 1 + len(fechas_futuras)).reshape(-1, 1)
        predicciones = model.predict(dias_futuros)
    else:
        # Para Prophet
        future = pd.DataFrame({'ds': fechas_futuras})
        forecast = model.predict(future)
        predicciones = forecast['yhat'].values
    
    df_futuro = pd.DataFrame({
        'Fecha': fechas_futuras,
        'Prediccion_KWh': predicciones
    })
    return df_futuro

# ========================= FUNCIÓN PRINCIPAL =========================
def comparar_modelos_diarios(df, col_fecha='Fecha', col_valor='Valor_KWh'):
    """
    Entrena todos los modelos con datos diarios, los compara y retorna predicciones hasta 2030.
    """
    print("="*70)
    print("ENTRENAMIENTO DE MODELOS CON DATOS DIARIOS")
    print("="*70)
    
    print("\nPreparando datos...")
    X, y, df_prep, fecha_inicio = preparar_datos_ml_diario(df, col_fecha, col_valor)
    df_prophet = preparar_datos_prophet_diario(df, col_fecha, col_valor)
    
    print(f"Total de registros: {len(X)} días")
    print(f"Fecha inicio: {fecha_inicio.date()}")
    print(f"Fecha fin datos: {df_prep['Fecha'].max().date()}")
    
    # Dividir en train/test (últimos 90 días para test)
    dias_test = 90
    split = len(X) - dias_test
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    df_prophet_train = df_prophet.iloc[:split]
    
    print(f"\nDatos de entrenamiento: {len(X_train)} días")
    print(f"Datos de prueba: {len(X_test)} días")
    
    resultados = {}
    
    # Random Forest
    print("\n[1/3] Entrenando Random Forest...")
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=15)
    rf_model.fit(X_train, y_train)
    rf_pred = rf_model.predict(X_test)
    resultados['Random Forest'] = {
        'modelo': rf_model,
        'metricas': {
            'MAE': mean_absolute_error(y_test, rf_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, rf_pred)),
            'R2': r2_score(y_test, rf_pred)
        },
        'tipo': 'ml'
    }
    
    # Regresión Lineal
    print("[2/3] Entrenando Regresión Lineal...")
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)
    lr_pred = lr_model.predict(X_test)
    resultados['Regresion Lineal'] = {
        'modelo': lr_model,
        'metricas': {
            'MAE': mean_absolute_error(y_test, lr_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, lr_pred)),
            'R2': r2_score(y_test, lr_pred)
        },
        'tipo': 'ml'
    }
    
    # Prophet
    print("[3/3] Entrenando Prophet...")
    prophet_model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        seasonality_mode='multiplicative'
    )
    prophet_model.fit(df_prophet_train)
    future_test = pd.DataFrame({'ds': df_prophet.iloc[split:]['ds']})
    prophet_forecast = prophet_model.predict(future_test)
    prophet_pred = prophet_forecast['yhat'].values
    resultados['Prophet'] = {
        'modelo': prophet_model,
        'metricas': {
            'MAE': mean_absolute_error(y_test, prophet_pred),
            'RMSE': np.sqrt(mean_squared_error(y_test, prophet_pred)),
            'R2': r2_score(y_test, prophet_pred)
        },
        'tipo': 'prophet'
    }
    
    # Mostrar resultados
    print("\n" + "="*70)
    print("RESULTADOS DE EVALUACIÓN (ÚLTIMOS 90 DÍAS)")
    print("="*70)
    for nombre, info in resultados.items():
        print(f"\n{nombre}:")
        for metrica, valor in info['metricas'].items():
            print(f"  {metrica}: {valor:.4f}")
    
    # Seleccionar mejor modelo (menor MAE)
    mejor_nombre = min(resultados, key=lambda x: resultados[x]['metricas']['MAE'])
    mejor_info = resultados[mejor_nombre]
    print(f"\n{'🏆'*35}")
    print(f"MEJOR MODELO: {mejor_nombre}")
    print(f"MAE: {mejor_info['metricas']['MAE']:.4f} KWh")
    print(f"RMSE: {mejor_info['metricas']['RMSE']:.4f} KWh")
    print(f"R²: {mejor_info['metricas']['R2']:.4f}")
    print(f"{'🏆'*35}")
    
    # Reentrenar con todos los datos y predecir hasta 2030
    print(f"\nReentrenando {mejor_nombre} con todos los datos...")
    if mejor_info['tipo'] == 'ml':
        if mejor_nombre == 'Random Forest':
            mejor_modelo_final = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=15)
        else:
            mejor_modelo_final = LinearRegression()
        mejor_modelo_final.fit(X, y)
    else:
        mejor_modelo_final = Prophet(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05,
            seasonality_mode='multiplicative'
        )
        mejor_modelo_final.fit(df_prophet)
    
    print("Generando predicciones hasta 2030-12-31...")
    df_predicciones = predecir_hasta_2030_diario(
        mejor_modelo_final, df_prep, fecha_inicio, mejor_info['tipo']
    )
    
    print(f"\nPredicciones generadas: {len(df_predicciones)} días")
    print("\nPrimeras 10 predicciones:")
    print(df_predicciones.head(10))
    print("\nÚltimas 10 predicciones (2030):")
    print(df_predicciones.tail(10))
    
    return mejor_nombre, mejor_modelo_final, df_predicciones, resultados

# ========================= USO =========================
# Para generación:
print("\n" + "█"*70)
print("MODELO PARA GENERACIÓN DE ENERGÍA")
print("█"*70)
mejor_gen, modelo_gen, pred_gen, res_gen = comparar_modelos_diarios(
    df_diario_generacion_gwh,
    col_fecha='Fecha',
    col_valor='Valor_KWh'
)

print("\n\n" + "█"*70)
print("MODELO PARA DEMANDA DE ENERGÍA")
print("█"*70)
# Para demanda:
'''mejor_dem, modelo_dem, pred_dem, res_dem = comparar_modelos_diarios(
    df_diario_demanda_gwh,
    col_fecha='Fecha',
    col_valor='Valor_KWh'
)

# Guardar predicciones en CSV
#pred_gen.to_csv('predicciones_generacion_2030.csv', index=False)
#pred_dem.to_csv('predicciones_demanda_2030.csv', index=False)

#print("\n✅ Predicciones guardadas en archivos CSV")

'''

01:33:41 - cmdstanpy - INFO - Chain [1] start processing



██████████████████████████████████████████████████████████████████████
MODELO PARA GENERACIÓN DE ENERGÍA
██████████████████████████████████████████████████████████████████████
ENTRENAMIENTO DE MODELOS CON DATOS DIARIOS

Preparando datos...
Total de registros: 1382 días
Fecha inicio: 2022-01-01
Fecha fin datos: 2025-10-13

Datos de entrenamiento: 1292 días
Datos de prueba: 90 días

[1/3] Entrenando Random Forest...
[2/3] Entrenando Regresión Lineal...
[3/3] Entrenando Prophet...


01:33:42 - cmdstanpy - INFO - Chain [1] done processing
01:33:42 - cmdstanpy - INFO - Chain [1] start processing
01:33:42 - cmdstanpy - INFO - Chain [1] done processing



RESULTADOS DE EVALUACIÓN (ÚLTIMOS 90 DÍAS)

Random Forest:
  MAE: 11203.5524
  RMSE: 13745.9223
  R2: -0.0008

Regresion Lineal:
  MAE: 11268.0293
  RMSE: 13801.5825
  R2: -0.0090

Prophet:
  MAE: 5705.1695
  RMSE: 7804.7498
  R2: 0.6773

🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆
MEJOR MODELO: Prophet
MAE: 5705.1695 KWh
RMSE: 7804.7498 KWh
R²: 0.6773
🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆🏆

Reentrenando Prophet con todos los datos...
Generando predicciones hasta 2030-12-31...

Predicciones generadas: 1905 días

Primeras 10 predicciones:
       Fecha  Prediccion_KWh
0 2025-10-14   237620.141185
1 2025-10-15   238375.621619
2 2025-10-16   238178.855154
3 2025-10-17   238203.384856
4 2025-10-18   227065.686112
5 2025-10-19   206892.718830
6 2025-10-20   229042.143934
7 2025-10-21   236842.264948
8 2025-10-22   237706.532063
9 2025-10-23   237594.922372

Últimas 10 predicciones (2030):
          Fecha  Prediccion_KWh
1895 2030-12-22   219987.576851
1896 2030-12-23   242749.233513
1897 2030-12-24   

'mejor_dem, modelo_dem, pred_dem, res_dem = comparar_modelos_diarios(\n    df_diario_demanda_gwh,\n    col_fecha=\'Fecha\',\n    col_valor=\'Valor_KWh\'\n)\n\n# Guardar predicciones en CSV\n#pred_gen.to_csv(\'predicciones_generacion_2030.csv\', index=False)\n#pred_dem.to_csv(\'predicciones_demanda_2030.csv\', index=False)\n\n#print("\n✅ Predicciones guardadas en archivos CSV")\n\n'